## Import stuff

In [1]:
import tensorflow as tf
from keras.losses import categorical_crossentropy, mean_squared_error
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras import layers
import shutil
import random
from PIL import Image
import numpy as np
import logging
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
import sklearn.metrics as metrics
from sklearn.svm import SVC
from sklearn import svm
from sklearn import datasets
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.kernel_ridge import KernelRidge
from sklearn.linear_model import Ridge, RidgeCV, ElasticNet
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.impute import KNNImputer
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import ClusterCentroids
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier, ExtraTreesClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingRegressor
logger = tf.get_logger()
logger.setLevel(logging.ERROR)
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

print(tf.__version__)

from tensorflow.python.client import device_lib
print("tensorflow doesn't detects GPU: ")
print(device_lib.list_local_devices() )
print(tf.test.gpu_device_name())
print(tf.config.list_physical_devices())

# print("torch detects GPU: " , torch.cuda.is_available())

oversampler = RandomOverSampler(random_state=0)
undersampler = ClusterCentroids(random_state=0)


def get_score(df_true, df_submission):
    for i in range(df_true.shape[1]):
        print("score for col ",i,": " , metrics.roc_auc_score(df_true[:,i], df_submission[:,i]))


2.8.0
tensorflow doesn't detects GPU: 
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6141285538938860660
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 6940786688
locality {
  bus_id: 1
  links {
  }
}
incarnation: 11692901367320746783
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1080, pci bus id: 0000:0b:00.0, compute capability: 6.1"
xla_global_id: 416903419
]
/device:GPU:0
[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


c:\ProgramData\Anaconda3\lib\site-packages\sklearn\experimental\enable_hist_gradient_boosting.py:16: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


## Load training data from csv


In [2]:
df = pd.read_csv("train_features_improved.csv")
labels_df = pd.read_csv("train_labels.csv")

X = np.array(df.iloc[:, 1:].values,  dtype=float)
Y = np.array(labels_df.iloc[:, 1:].values,  dtype=float)

#normalize
# Y[:,13] = (Y[:,13]-27)/73

X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2, random_state=0)

print("X shape: ", X.shape)
print("Y shape: ", Y.shape)



X shape:  (18995, 546)
Y shape:  (18995, 15)


## The function that trains and rates models

In [3]:
def find_best_model(col, X_train, Y_train, X_valid, Y_valid, X_test, X, Y):
    models = []

    Y_train = Y_train[:,col]
    Y_valid = Y_valid[:,col]
    # print(X_train.shape)
    # X_train = PolynomialFeatures(degree=2).fit_transform(X_train)
    # print(X_train.shape)
    # X_valid = PolynomialFeatures(degree=2).fit_transform(X_valid)
    # X_train = SelectKBest(chi2, k=35).fit_transform(X_train+0.01, Y_train)
    # X_valid = SelectKBest(chi2, k=35).fit_transform(X_valid+0.01, Y_valid)
    # X_train, Y_train = oversampler.fit_resample(X_train, Y_train)
    # X_train, Y_train = undersampler.fit_resample(X_train, Y_train)

    #task 1 and 2
    if(col<11):    
        # print("training improved neural net for col ", col)
        # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=3)
        # model = tf.keras.Sequential([
        #     layers.Dense(512, activation=tf.keras.activations.relu),
        #     # layers.BatchNormalization(),
        #     layers.Dense(512, activation=tf.keras.activations.relu),
        #     # layers.BatchNormalization(),
        #     layers.Dense(1, activation='sigmoid')
        # ])
        # model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001))
        # history = model.fit(X_train, Y_train, epochs=100, verbose=False,callbacks=[callback], validation_data=(X_valid, Y_valid))
        # print(len(history.history['loss']))
        # models.append(model)

        # print("training ridge",1," for col ", col)
        # ridge = Ridge()
        # ridge.fit(X_train, Y_train)
        # models.append(ridge)
        
        print("training HistGradientBoostingClassifier for col ", col)
        model = HistGradientBoostingClassifier()
        model.fit(X_train, Y_train)
        models.append(model)
        
        print("training ExtraTrees for col ", col)
        model = ExtraTreesClassifier(n_estimators=100, random_state=0)
        model.fit(X_train, Y_train)
        models.append(model)

        print("training rfc for col ", col)
        rfc = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=10)
        rfc.fit(X_train, Y_train)
        models.append(rfc)

        # print("training decision tree for col ", col)
        # model = DecisionTreeClassifier(max_depth=5)
        # model.fit(X_train, Y_train)
        # models.append(model)

        # print("training adaboost for col ", col)
        # adaboost = AdaBoostClassifier(n_estimators=100, random_state=10)
        # adaboost.fit(X_train, Y_train)
        # models.append(adaboost)

        # print("training GradientBoostingClassifier for col ", col)
        # model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1, max_depth=1, random_state=0)
        # model.fit(X_train, Y_train)
        # models.append(model)


    # task 2
    # if(col==10):
    #     print("training improved neural net for col ", col)
    #     callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
    #     model = tf.keras.Sequential([
    #         layers.Dense(512, activation=tf.keras.activations.relu),
    #         layers.Dense(512, activation=tf.keras.activations.relu),
    #         layers.Dense(1, activation='sigmoid')
    #     ])
    #     model.compile(loss='binary_crossentropy', optimizer=tf.keras.optimizers.Adam(0.001))
    #     history = model.fit(X_train, Y_train, epochs=100, verbose=False,callbacks=[callback])
    #     print(len(history.history['loss']))
    #     models.append(model)

    #     print("training ridge 1 for col ", col)
    #     ridge = Ridge(alpha=1.0)
    #     ridge.fit(X_train, Y_train)
    #     models.append(ridge)
    
    #task 3
    if(col>=11):
        # print("training improved neural net for col ", col)
        # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=3, restore_best_weights=True)
        # model = tf.keras.Sequential([
        #     layers.Dense(512, activation=tf.keras.activations.relu),
        #     layers.Dense(512, activation=tf.keras.activations.relu),
        #     layers.Dense(1, activation='linear')
        # ])
        # model.compile(loss="mean_squared_error", optimizer=tf.keras.optimizers.Adam(0.001))
        # history = model.fit(X_train, Y_train, epochs=100, verbose=False,callbacks=[callback], validation_data=(X_valid, Y_valid))
        # print(len(history.history['loss']))
        # models.append(model)

        # print("training ridge for col ", col)
        # ridge = Ridge(alpha=1.0)
        # ridge.fit(X_train, Y_train)
        # models.append(ridge)

        print("training HistGradientBoostingRegressor for col ", col)
        model = HistGradientBoostingRegressor()
        model.fit(X_train, Y_train)
        models.append(model)




    
    print("done training all models for col ", col)

    best_score = 0
    best_model = None
    probabilistic_models = ["MLPClassifier", "SVC", "RandomForestClassifier", "HistGradientBoostingClassifier", "GradientBoostingClassifier", "ExtraTreesClassifier", "RandomForestRegressor", "DecisionTreeClassifier", "GaussianNB", "AdaBoostClassifier", "QuadraticDiscriminantAnalysis", "GaussianProcessClassifier"]
    for model in models:
        model_type = str(type(model)).split(".")[-1].split("'")[0]
        # print("testing model ", model_type)
        #if the model is probabilistic
        if model_type in probabilistic_models:
            pred = model.predict_proba(X_valid)[:,1]
        else:
            pred = model.predict(X_valid)
        if(col<11):  
            np.clip(pred, 0.0, 1.0, out=pred)
            # print(pred.min())
            # print(pred.max())
        # print("predictions shape: ", pred.shape)
        if(col>=11):
            score = 0.5 + 0.5 * np.maximum(0, metrics.r2_score(Y_valid, pred))
        else:
            score = metrics.roc_auc_score(Y_valid, pred)
        print("score for",type(model),": " , score)
        if score > best_score:
            best_score = score
            best_model = model

    print("best model: ", type(best_model),best_score)

    #doesn't seem to work
    # if str(type(best_model)).count("Sequential") > 0:
    #     best_model.fit(X,Y[:,col], epochs=15, verbose=False)
    # else:
    #     best_model.fit(X, Y[:,col])

    predictions = best_model.predict(X_test)
    # print("predictions shape: ", predictions.shape)

    return predictions,best_score

## Writing results

In [4]:
X_test = pd.read_csv("test_features_improved.csv")
X_test = np.array(X_test.iloc[:, 1:].values,  dtype=float)

data = []
scores = []

for i in range(0,15):
# for i in range(10,11):
# for i in range(11,15):
# for i in range(13,14):
    # if i == 11:
    #     X = np.array(pd.read_csv("train_features_imputed.csv").iloc[:, 1:].values,  dtype=float)
    #     X_test = np.array(pd.read_csv("test_features_imputed.csv").iloc[:, 1:].values,  dtype=float)

    # pred = find_best_model(i, X, Y, X, Y, X_test,X,Y)
    pred,best_score = find_best_model(i, X[:int(X.shape[0]*0.8)], Y[:int(Y.shape[0]*0.8)], X[int(X.shape[0]*0.8):], Y[int(Y.shape[0]*0.8):], X_test,X,Y)
    # pred = find_best_model(i, X[int(X.shape[0]*0.2):], Y[int(Y.shape[0]*0.2):], X[:int(X.shape[0]*0.2)], Y[:int(Y.shape[0]*0.2)], X_test,X,Y)
    # print("pred shape: ", pred.shape)
    pred = pred.reshape(pred.shape[0],1)
    data.append(pred)
    scores.append(best_score)





training HistGradientBoostingClassifier for col  0
training ExtraTrees for col  0
training rfc for col  0
done training all models for col  0
score for <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'> :  0.9294584997167452
score for <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> :  0.925518321749824
score for <class 'sklearn.ensemble._forest.RandomForestClassifier'> :  0.9268525555110332
best model:  <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'> 0.9294584997167452
training HistGradientBoostingClassifier for col  1
training ExtraTrees for col  1
training rfc for col  1
done training all models for col  1
score for <class 'sklearn.ensemble._hist_gradient_boosting.gradient_boosting.HistGradientBoostingClassifier'> :  0.8059231427174977
score for <class 'sklearn.ensemble._forest.ExtraTreesClassifier'> :  0.7991309261974584
score for <class 'sklearn.ensemble._forest.RandomForest

In [5]:
print(np.mean(scores[:10]))
print(np.mean(scores[10]))
print(np.mean(scores[11:]))

print(np.mean([np.mean(scores[:10]),np.mean(scores[10]),np.mean(scores[11:])]))


0.8201741012508948
0.6848831672158041
0.7693663987946994
0.7581412224204661


In [6]:
np_array = np.array(data).transpose()[0]

#un-normalize
# np_array[:,13] = (np_array[:,13]*73)+27

col_names = ['LABEL_BaseExcess', 'LABEL_Fibrinogen', 'LABEL_AST', 'LABEL_Alkalinephos', 'LABEL_Bilirubin_total', 'LABEL_Lactate', 'LABEL_TroponinI', 'LABEL_SaO2', 'LABEL_Bilirubin_direct', 'LABEL_EtCO2', 'LABEL_Sepsis', 'LABEL_RRate', 'LABEL_ABPm', 'LABEL_SpO2', 'LABEL_Heartrate']
print(len(col_names))
new_df = pd.DataFrame(np_array, columns=col_names)

output_csv = pd.read_csv("sample.csv")
output_csv.update(new_df)
output_csv.to_csv('submit.zip', index=False, float_format='%.3f', compression='zip')
output_csv.to_csv('submit.csv', index=False, float_format='%.3f')


15
